<a href="https://colab.research.google.com/github/Darksoule123/COURSEAREA-TEST1/blob/master/FAB_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langgraph langchain_community tavily-python langchain-core langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/

In [19]:
#restricted search

import time
from typing_extensions import TypedDict
from typing import Annotated
from langgraph.graph.message import add_messages
from langchain.schema import HumanMessage  # Import HumanMessage to check type

# Define the State class
class State(TypedDict):
    messages: Annotated[list, add_messages]

# Initialize the StateGraph
from langgraph.graph import StateGraph, START, END
graph_builder = StateGraph(State)

# Set up API keys
import os
from tavily import TavilyClient
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool, AgentType

# Assuming you're working on Google Colab, you can use `get` from `google.colab` for user data.
from google.colab import userdata

openaiapikey = userdata.get('openaiapikey').strip()
apikey = userdata.get('api_key')

os.environ["OPENAI_API_KEY"] = openaiapikey
os.environ["TAVILY_API_KEY"] = apikey

# Initialize the language model
llm = ChatOpenAI(
    model="gpt-4",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=10,
    api_key=openaiapikey
)

# Custom function to restrict search to multiple sites
def restricted_tavily_search(query: str, sites: list) -> str:
    site_query = " OR ".join([f"site:{site}" for site in sites])  # Combine sites with OR
    restricted_query = f"{site_query} {query}"  # Prepend the site restriction to the query
    return tool.run(restricted_query)

# Define the sites you want to restrict the search to
restricted_sites = ["indiamart.com", "amazon.in", "flipkart.com"]

# Create a Tool object for LangChain agent with the restricted search function
search_tool = Tool(
    name="Restricted Search",
    func=lambda query: restricted_tavily_search(query, restricted_sites),  # Include the sites list directly
    description="Searches using Tavily for information from specified sites."
)

# Initialize the agent with the language model and tools
tools = [search_tool]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Define the chatbot function using the agent
def chatbot(state: State):
    last_message = state["messages"][-1]  # Get the last message object
    if isinstance(last_message, HumanMessage):  # Ensure it's a HumanMessage
        user_input = last_message.content  # Access the content
    else:
        user_input = last_message["content"]  # Fallback for dictionary-like access

    response = agent.run(user_input)  # Get the response from the agent
    state["messages"].append({"role": "assistant", "content": response})  # Add response to the state
    return state

# Build the graph
from langgraph.prebuilt import ToolNode, tools_condition

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[search_tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

# Initialize memory saver
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

# Compile the graph
graph = graph_builder.compile(checkpointer=memory)

# Configuration
config = {"configurable": {"thread_id": "1"}}

# Main loop for user interaction
state = {"messages": []}  # Initialize the state
while True:
    user_input = input("User: ")  # Capture user input
    if user_input.lower() == "q":  # Exit condition
        print("Goodbye!")
        break

    state["messages"].append({"role": "user", "content": user_input})  # Add user input to state

    start_time = time.time()  # Start time tracking

    # Process the user input and generate response
    state = chatbot(state)  # Pass state to chatbot function

    # Print the assistant's response
    print(f"Assistant: {state['messages'][-1]['content']}")
    print("--")

    end_time = time.time()  # End time tracking
    print(f"Time taken for this interaction: {end_time - start_time:.2f} seconds")



User: I want to buy an iphone 14 what is the best site to buy it 


> Entering new AgentExecutor chain...
The iPhone 14 has not been released yet, so it's not possible to buy it from any site. 
Final Answer: The iPhone 14 is not available for purchase as it has not been released yet.

> Finished chain.
Assistant: The iPhone 14 is not available for purchase as it has not been released yet.
--
Time taken for this interaction: 1.81 seconds
User: i want to buy an iphone 13 


> Entering new AgentExecutor chain...
The user is looking for a place to buy an iPhone 13. I should find reputable online stores where the iPhone 13 is available for purchase.
Action: Restricted Search
Action Input: Buy iPhone 13 online
Observation: [{'url': 'https://www.flipkart.com/apple-iphone-13-product-red-128-gb/p/itm99b5658d148b0?pid=MOBG6VF59ZFEPEBX&marketplace=FLIPKART', 'content': 'Rating  4.6  (13,705) Buy Apple iPhone 13 online at best price with offers in India. Apple iPhone 13 (Red, 128 GB) with features

In [ ]:
#no restricted search

import time
from typing_extensions import TypedDict
from typing import Annotated
from langgraph.graph.message import add_messages
from langchain.schema import HumanMessage  # Import HumanMessage to check type

# Define the State class
class State(TypedDict):
    messages: Annotated[list, add_messages]

# Initialize the StateGraph
from langgraph.graph import StateGraph, START, END
graph_builder = StateGraph(State)

# Set up API keys
import os
from tavily import TavilyClient
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool, AgentType

# Assuming you're working on Google Colab, you can use `get` from `google.colab` for user data.
from google.colab import userdata

openaiapikey = userdata.get('openaiapikey').strip()
apikey = userdata.get('api_key')

os.environ["OPENAI_API_KEY"] = openaiapikey
os.environ["TAVILY_API_KEY"] = apikey

# Initialize the language model
llm = ChatOpenAI(
    model="gpt-4",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=10,
    api_key=openaiapikey
)

# Set up the search tool
from langchain_community.tools.tavily_search import TavilySearchResults
tool = TavilySearchResults(max_results=10)

# Create a Tool object for LangChain agent
search_tool = Tool(
    name="Search",
    func=tool.run,
    description="Searches using Tavily for information."
)

# Initialize the agent with the language model and tools
tools = [search_tool]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Define the chatbot function using the agent
def chatbot(state: State):
    last_message = state["messages"][-1]  # Get the last message object
    if isinstance(last_message, HumanMessage):  # Ensure it's a HumanMessage
        user_input = last_message.content  # Access the content
    else:
        user_input = last_message["content"]  # Fallback for dictionary-like access

    response = agent.run(user_input)  # Get the response from the agent
    state["messages"].append({"role": "assistant", "content": response})  # Add response to the state
    return state

# Build the graph
from langgraph.prebuilt import ToolNode, tools_condition

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

# Initialize memory saver
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

# Compile the graph
graph = graph_builder.compile(checkpointer=memory)

# Configuration
config = {"configurable": {"thread_id": "1"}}

# Main loop for user interaction
state = {"messages": []}  # Initialize the state
while True:
    user_input = input("User: ")  # Capture user input
    if user_input.lower() == "q":  # Exit condition
        print("Goodbye!")
        break

    state["messages"].append({"role": "user", "content": user_input})  # Add user input to state

    start_time = time.time()  # Start time tracking

    # Process the user input and generate response
    state = chatbot(state)  # Pass state to chatbot function

    # Print the assistant's response
    print(f"Assistant: {state['messages'][-1]['content']}")
    print("--")

    end_time = time.time()  # End time tracking
    print(f"Time taken for this interaction: {end_time - start_time:.2f} seconds")
